# Practical Part: Neural Network Implementation & Experiments

Load the Fashion MNIST data:
Note: keep your file structures like this for reading input data without
using ```import os``` for path change!
```
./Homework 3
├── 3_practical_part.ipynb
├── circles.txt
├── data
│   ├── fashion
│   │   ├── t10k-images-idx3-ubyte.gz
│   │   ├── t10k-labels-idx1-ubyte.gz
│   │   ├── train-images-idx3-ubyte.gz
│   │   └── train-labels-idx1-ubyte.gz
│   └── mnist
│       └── README.md
├── hw3
│   └── d3english.pdf
├── overleaf_url.txt
└── utils
    ├── __init__.py
    ├── __pycache__
    │   ├── __init__.cpython-36.pyc
    │   └── mnist_reader.cpython-36.pyc
    ├── argparser.py
    ├── helper.py
    └── mnist_reader.py
```

In [1]:
import utils.mnist_reader as mnist_reader
import numpy as np
import math
import copy 

X_train, y_train = mnist_reader.load_mnist('data/fashion', kind='train')
X_test, y_test = mnist_reader.load_mnist('data/fashion', kind='t10k')

print(X_test.shape)

(10000, 784)


Load the Circles data:

In [189]:
circlesData = np.loadtxt(open('circles.txt','r'))
circlesTarget = circlesData[:,2]
circlesData = circlesData[:,[0,1]] 
#circlesData = circlesData
#circlesData = np.expand_dims(circlesData, axis = 1)
#circlesData = circlesData.reshape(1100, 2, 1)
print(circlesData.shape)
circlesTarget = np.array([int(i) for i in circlesTarget])
print(circlesTarget)

(1100, 2)
[1 1 0 ... 0 1 1]


In [3]:
class loadData:
    def __init__(self):
        self.addOnes = False
        self.data_path = '/data/'
    
    def convertTarget(self, targetValues):
        # Convert to one-hot encoding
        numClasses = np.max(targetValues) + 1
        return np.eye(numClasses)[targetValues]
    

    def loadNumData(self, data, target):
        # Split into train/validation/test
        np.random.seed(6390)
        randIndices = np.random.permutation(data.shape[0])
        data, target = data[randIndices], target[randIndices]
        
        div1 = int(math.floor(0.8 * data.shape[0]))
        div2 = int(math.floor(0.9 * data.shape[0]))
        trainData, trainTarget = data[:div1], target[:div1]
        validData, validTarget = data[div1:div2], target[div1:div2]
        testData, testTarget = data[div2:], target[div2:]
    
        # Get one hot encoding
        trainTarget = self.convertTarget(trainTarget)
        validTarget = self.convertTarget(validTarget)
        testTarget = self.convertTarget(testTarget)
        
        return trainData, trainTarget, validData, validTarget, testData, testTarget

dataLoader = loadData()
 
trainData, trainTarget, validData, validTarget, testData, testTarget = dataLoader.loadNumData(circlesData, circlesTarget)
print(trainData.shape, validData.shape, testData.shape)




(880, 2, 1) (110, 2, 1) (110, 2, 1)


## Experiments

### Part 1

> As a beginning, start with an implementation that computes the gradients for a single example, and check that the gradient is correct using the finite difference method described above.

In [4]:
class BatchSampler(object):
    '''
    A (very) simple wrapper to randomly sample batches without replacement.
    '''
    
    def __init__(self, data, targets, batch_size):
        self.num_points = data.shape[0]
        self.features = data.shape[1]
        self.data = data
        self.targets = targets
        self.batch_size = batch_size
        self.indices = np.arange(self.num_points)

    def random_batch_indices(self, m=None):
        if m is None:
            indices = np.random.choice(self.indices, self.batch_size, replace=False)
        else:
            indices = np.random.choice(self.indices, m, replace=False)
        return indices 

    def get_batch(self, m=None):
        '''
        Get a random batch without replacement from the dataset.
        If m is given the batch will be of size m. 
        Otherwise will default to the class initialized value.
        '''
        indices = self.random_batch_indices(m)
        X_batch = np.take(self.data, indices, 0)
        y_batch = self.targets[indices]
        return X_batch, y_batch

In [214]:
# Our own activation functions

def relu(pre_activation):
    '''
    preactivation is a vector
    '''
    relu_output = np.zeros(pre_activation.shape)
    #print("relu_output ", relu_output)
    relu_flat = relu_output.flatten()
    #print("relu_flat ", relu_flat)
    
    #print(pre_activation.flatten())
    for i, neuron in enumerate(pre_activation.flatten()):
        if neuron > 0:
            relu_flat[i] = neuron
    relu_output = relu_flat.reshape(pre_activation.shape)
    return relu_output

def softmax(pre_activation):
    '''
    Numerically stable because subtracting the max value makes bit overflow impossible,
    we will only have non-positive values in the vector
    '''
    #print("*********in softmax**************")
    #print("preactivation", pre_activation)
    exps = np.exp(pre_activation - np.max(pre_activation))
    #print("exps \n", exps)
    #print(" exps / np.sum(exps) \n",  exps / np.sum(exps))
    
    return exps / np.sum(exps)

#relu(np.array([[ 0.81520074, -0.17279209],
# [ 0.81520074, -0.17279209]]))

In [617]:
class neuralNet():
    def __init__(self, d, dh, m, eta=1, regularize=None):
        self.inputDim = d #inputDim
        self.hiddenDim = dh #hiddenDim
        self.outputDim = m #outputDim
        self.regularize = regularize # lambda value
        self.learningRate = eta
        
        self.batchErrorGradients = []
        
        
        #may use xavier init - maybe explore this later.
        
        # Initial weights and biases
        #while self.W_1 == 0:
        self.W_1 = np.random.uniform(-1/np.sqrt(d), 1/np.sqrt(d), d*dh).reshape(dh, d)
        self.W_2 = np.random.uniform(-1/np.sqrt(dh), 1/np.sqrt(dh), dh*m).reshape(m, dh) 
        self.b_1 = np.zeros(dh).reshape(dh,)
        self.b_2 = np.zeros(m).reshape(m,)
        
        print("check init weights")
        print("self.W_1 ", self.W_1)
        print("self.W_2 ", self.W_2)
        
    def fprop(self, batchData, batchTarget):
        #print("in fprop, ", self.W_1.shape, x.shape, self.b_1.shape)
        #print("batchData.shape ", batchData.shape)
        #print("np.dot(self.W_1, batchData).shape ", np.dot(self.W_1, batchData).shape)
        self.h_a = np.dot(self.W_1, batchData) + self.b_1
        #print('h_a (input to RELU)', self.h_a)
        
        # TODO: may come back to expand dim for single point
        #self.h_s = np.expand_dims(relu(self.h_a), axis = 1)
        self.h_s = relu(self.h_a)
        #print('h_s (output of RELU)', self.h_s)
        #print("self.h_a.shape", self.h_a.shape)
        #print("self.h_s.shape", self.h_s.shape)

        self.o_a = np.dot(self.W_2, self.h_s) + self.b_2
        #print('self.o_a', self.o_a)
        self.o_s = softmax(self.o_a)
        #print('o_s', self.o_s)
        #print('batchTarget', batchTarget)
        
        # Update list of error gradients (to be averaged)
        #error_gradient = self.o_s - batchTarget
        error_gradient = np.subtract(self.o_s, batchTarget)
        #print('error_gradient:', error_gradient)
        self.batchErrorGradients.append(error_gradient)
        
        
    def errorRate(self, y):
        '''
        negative log
        -logO_s(x)
        '''
        negLog = -self.o_a[y] + np.log(np.sum(np.exp(self.o_a), axis=0))
        
        #print('returning this error:', error)
        return negLog
            
    def bprop(self, batchData):
        '''
        batchTarget already in one-hot format
        '''

        #print("self.o_s", self.o_s)
        #print("self.grad_oa.shape", self.grad_oa)
        #print("self.errorRate() ", self.errorRate())
        
        # the np.mean is taking avg of loss
        # pack prop taking the avg to multiply over all the grads
        #self.grad_oa = (self.grad_oa, self.errorRate())
        #print('error_gradients_list:', self.batchErrorGradients)
        
        #print("batch data before start to back prop ", batchData.shape)
        
        self.grad_oa = np.mean(np.array(self.batchErrorGradients), axis=0)
        #print("shape before hack ", self.grad_oa.shape)
        if batchData.shape[0] == 1: # Batch size 1
            self.grad_oa = np.expand_dims(self.grad_oa, axis = 1)
            #print("inside hack dim change ", self.grad_oa.shape)

        if batchData.shape[0] == 1:
            self.h_s = np.expand_dims(self.h_s, axis = 1)
            #self.grad_oa = np.expand_dims(self.grad_oa, axis = 1)
            #print("self.grad_oa", self.grad_oa.shape)    
            #print("****** self.h_s *****", self.h_s.shape)
        self.grad_W2 = np.dot(self.grad_oa, self.h_s.T)
        
        #print("******** in back prop ********")
        #print('self.grad_W2', self.grad_W2)
        self.grad_b2 = self.grad_oa
        #print('grad_b2', self.grad_b2)
        self.grad_hs = np.dot(self.grad_oa.T, self.W_2)
        #print('grad_hs', self.grad_hs)
        
        # Check this (dim mismatch maybe)
        #print("self.h_a ", self.h_a.shape)
        #print("(np.where(self.h_a > 0, 1, 0)))", (np.where(self.h_a > 0, 1, 0)).shape)
        
        
        print("self.grad_hs ", self.grad_hs.shape)
        
        # grad_hs should be dh by 1
        print("self.h_a ", self.h_a)
        print("npwhere ", np.where(self.h_a > 0, 1, 0))
        
        #h_a_stack = np.expand_dims(np.where(self.h_a > 0, 1, 0), axis=0)
        h_a_stack = np.where(self.h_a > 0, 1, 0)
        print("h_a_stack ", h_a_stack)
        self.grad_ha =  h_a_stack  * self.grad_hs
        print('grad_ha', self.grad_ha.shape)
        
        #np.dot(np.expand_dims(self.grad_oa, axis=1), np.expand_dims(self.h_s, axis=1).T)
        
        print("batchData before grad_W1", batchData.shape)
        #if batchData.shape[0] == 1:
        #    batchData = np.expand_dims(batchData, axis = 1)
        self.grad_W1 = np.dot(self.grad_ha.T, batchData)
        
        # temporary hack for grad_W1
        #self.grad_W1 = self.grad_W1.T
        
        print('grad_W1', self.grad_W1)
        self.grad_b1 = self.grad_ha
        #self.grad_b1 = self.grad_b1.T

        print('grad_b1', self.grad_b1)
        
        self.grad_reg1 = 0# regularization for layer 1
        self.grad_reg2 = 0# regularization for layer 2
    
    def updateParams(self):
        #print("********in update param********")
        #print("self.grad_W1 ", self.grad_W1)
        #print("self.W_1 ", self.W_1)
        self.W_1 -= np.sum(self.grad_W1, axis = 0) * self.learningRate
        self.W_2 -= self.grad_W2 * self.learningRate
        #print("grad_b1 ", self.grad_b1.shape)
        
        self.b_1 -= self.grad_b1 * self.learningRate
        self.b_2 -= self.grad_b2 * self.learningRate
    
    def gradDescentLoop(self, batchData, batchTarget, K):
        # Call each example in the data (over the minibatches) in a loop
        
        
        if K == 1: # For batch size = 1 (one example)
            self.fprop(batchData, batchTarget)
            
            batchDataBack = np.expand_dims(batchData, axis = 0)
            print('batchData shape in grad descent loop, k=1', batchDataBack)
            self.bprop(batchDataBack)
        else: # For minibatch with size K
            running_error = 0
            for i in range(K):
                print( "batchData[i]", batchData[i].shape)
                self.fprop(batchData[i], batchTarget[i])
            print("grad loop multi data", batchData.shape)
            self.bprop(batchData)
            self.updateParams()
            
            
    '''def gradDescentMat():
        # Feed the entire data matrix in as input
        miniBatch = sample(K indices)
        #bprop using matrix operations
        
    '''

        
    def dataSplit(self):
        '''
        train
        test
        valid
        '''
            
'''      
def earlyStopping(net):
    totalEpoch = 10 #may not be enough??
    for each epoch
        train
        valid
        test
        plot / print errors
'''



'      \ndef earlyStopping(net):\n    totalEpoch = 10 #may not be enough??\n    for each epoch\n        train\n        valid\n        test\n        plot / print errors\n'

In [618]:
class debugNet():
    
    def __init__(self, originalNet):
        self.copyNet = copy.deepcopy(originalNet)
        self.originalNet = originalNet
        self.idxList = ['W2', 'b2', 'W1', 'b1']
        self.originalParamList = {'W2': originalNet.W_2, 'b2': originalNet.b_2, 
                             'W1': originalNet.W_1, 'b1': originalNet.b_1}
        self.copyParamList = {'W2': self.copyNet.W_2, 'b2': self.copyNet.b_2, 
                              'W1': self.copyNet.W_1, 'b1': self.copyNet.b_1}
        self.originalGradientList = {'W2': originalNet.grad_W2, 'b2': originalNet.grad_b2,
                                    'W1': originalNet.grad_W1, 'b1': originalNet.grad_b1}
        
    def finiteGradCheck(self, sigma, x, y):
        #for i, param in enumerate(copyParamList):
        y = np.argmax(y)
        print("y at top of debug ", y)

        for key in self.idxList: #, param in self.copyParamList.items():
            param = self.copyParamList[key]
            print('key:', key, 'param:', param)
            
            '''if x.ndim == 1:
                originalShape = self.originalParamList[key].shape
                flatParams = np.ndarray.flatten(param)
                print('flattened parameters:', flatParams)
                for j, item in enumerate(flatParams):
                    print('param j:', item)
                    self.copyParamList[key][j] += sigma
                    self.copyNet.fprop(x, y)
                    originalScalar = self.originalParamList[key][j]
                    grad_estimate = ( self.copyNet.errorRate(y) - self.originalNet.errorRate(y)) / sigma
                    #print('self.originalGradientList', self.originalGradientList)
                    print("self.originalGradientList[key] ", self.originalGradientList[key])
                    originalGradient = self.originalGradientList[key][j]
                    ratio = originalGradient / grad_estimate
                    print('The ratio of calculated gradient and estimated gradient by perturbing {} gives {}.'.format(key, ratio))
                    # Set back
                    self.copyParamList[key][j] = originalScalar'''
            if 1==1:
                flatCopyParam = np.ndarray.flatten(param)
                flatOriginalParam = np.ndarray.flatten(self.originalParamList[key])
                originalShape = self.originalParamList[key].shape
                for j, _ in enumerate(flatCopyParam):
                    jRow = j % originalShape[0]
                    jCol = j // originalShape[0]
                    print("original shape", originalShape)
                    print("j % originalShape[0] = {}, j // originalShape[0] = {}".format(j % originalShape[0],j // originalShape[0]))
                    
                    
                    # Perturb
                    if param.ndim > 1:
                        self.copyParamList[key][jRow][jCol] += sigma
                    else:
                        self.originalParamList[key][jCol] +=sigma
                    self.copyNet.fprop(x, y)
                    
                    if param.ndim > 1:
                        originalScalar = self.originalParamList[key][jRow][jCol]
                    else:
                        originalScalar = self.originalParamList[key][jCol]
                    grad_estimate = ( self.copyNet.errorRate(y) - self.originalNet.errorRate(y)) / sigma
                    
                    if param.ndim > 1:
                        originalGradient = self.originalGradientList[key][jRow][jCol]
                    else:
                        originalGradient = self.originalGradientList[key][jCol]
                    ratio = originalGradient / grad_estimate
                    print('The ratio of calculated gradient and estimated gradient by perturbing {} gives \
                            {}.'.format(key, ratio))
                    print('original gradient', originalGradient)
                    print('grad_estimate ', grad_estimate)
                    # Set back
                    if param.ndim > 1:
                        self.copyParamList[key][jRow][jCol] = originalScalar
                    else:
                        self.copyParamList[key][jCol] = originalScalar
                

In [620]:
# Do forward and backprop for one example

# Initialize net
print( trainData[0:1][0].shape)


nn = neuralNet(2, 7, 2)
nn.gradDescentLoop(trainData[0:1][0],  trainTarget[0], 1)

# Print the gradients
print('W1 gradient: \n{} \n\n b1 gradient:\n{}'.format(nn.grad_W1, nn.grad_b1))
print("softmax result: \n {}".format(nn.o_s.shape))

finiteGrad = debugNet(nn)

# single input add one dim hack
#print("trainTarget[0:]", trainTarget[0:1].T.shape)

finiteGrad.finiteGradCheck(0.000001, trainData[0:1][0], trainTarget[0])



(2,)
check init weights
self.W_1  [[-0.48676376  0.19880266]
 [-0.11415313  0.3425119 ]
 [-0.07376937 -0.21425601]
 [-0.60174902 -0.65586917]
 [ 0.31009027 -0.61465458]
 [ 0.34792068 -0.33004544]
 [ 0.442288    0.56905413]]
self.W_2  [[ 0.22150354 -0.23572617  0.00401841  0.30061789  0.08781    -0.03155619
   0.0286677 ]
 [-0.31784683  0.01380136 -0.29710985 -0.25296633  0.32469797  0.32319154
  -0.13093883]]
batchData shape in grad descent loop, k=1 [[ 0.82868251 -0.55971895]]
self.grad_hs  (1, 7)
self.h_a  [-0.51464623 -0.2863071   0.05879176 -0.13155648  0.6010002   0.47304846
  0.04800595]
npwhere  [0 0 1 0 1 1 1]
h_a_stack  [0 0 1 0 1 1 1]
grad_ha (1, 7)
batchData before grad_W1 (1, 2)
grad_W1 [[-0.          0.        ]
 [ 0.         -0.        ]
 [-0.14241898  0.09619438]
 [-0.          0.        ]
 [ 0.11203646 -0.07567305]
 [ 0.16777837 -0.11332294]
 [-0.0754861   0.05098575]]
grad_b1 [[-0.          0.         -0.17186193 -0.          0.13519829  0.202464
  -0.09109171]]
W1 gra

/Users/vikuo/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:68: RuntimeWarning: invalid value encountered in double_scalars
/Users/vikuo/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:68: RuntimeWarning: divide by zero encountered in true_divide


### Part 2

> Display  the  gradients  for  both  methods (direct computation and finite difference) for a small network (e.g. $d = 2$ and $d_{h} = 2$) with random weights and for a single example.

### Part 3

> Add a hyperparameter for the minibatch size $K$ to allow computing the gradients on a minibatch of $K$ examples (in a matrix), by looping over the $K$ examples (this is a small addition to your previous code).

### Part 4

> Display the gradients for both methods (direct computation and finite difference) for a small network (e.g. $d = 2$ and $d_{h} = 2$) with random weights and for a minibatch with 10 examples (you can use examples from both classes from the two circles dataset).

In [474]:
#trainData = np.squeeze(trainData, axis = 2)

print(trainTarget[0:10].shape)

nn = neuralNet(2, 2, 2)
nn.gradDescentLoop((trainData[0:10]), trainTarget[0:10], 10)

(10, 2)
check init weights
self.W_1  [[ 0.19695501  0.23655265]
 [ 0.29935929 -0.23823049]]
self.W_2  [[ 0.14221396 -0.5444647 ]
 [-0.63487806  0.62743014]]
batchData[i] (2,)
batchData[i] (2,)
batchData[i] (2,)
batchData[i] (2,)
batchData[i] (2,)
batchData[i] (2,)
batchData[i] (2,)
batchData[i] (2,)
batchData[i] (2,)
batchData[i] (2,)
grad loop multi data (10, 2)
******** in back prop ********
self.grad_W2 0.05758266776674969


ValueError: shapes (2,) and (10,2) not aligned: 2 (dim 0) != 10 (dim 0)

### Part 5

> Train your neural network using gradient descent on the two circles dataset. Plot the decision regions for several different values of the hyperparameters (weight decay, number of hidden units, early stopping) so as to illustrate their effect on the capacity of the model.

### Part 6

> As a second step, copy your existing implementation to modify it to a new implementation that will use matrix calculus (instead of a loop) on batches of size $K$ to improve efficiency. **Take the matrix expressions in numpy derived in the first part, and adapt them for a minibatch of size $K$. Show in your report what you have modified (describe the former and new expressions with the shapes of each matrix).**

### Part 7

> Compare both implementations (with a loop and with matrix calculus) to check that they both give the same values for the gradients on the parameters, first for $K = 1$, then for $K = 10$. Display the gradients for both methods.

### Part 8

> Time how long an epoch takes on Fashion MNIST (1 epoch = 1 full traversal through the whole training set) for $K = 100$ for both versions (loop over a minibatch and matrix caluclus).

### Part 9

> Adapt your code to compute the error (proportion of misclassified examples) on the training set as well as the total loss on the training set during each epoch of the training procedure, and at the end of each epoch, it computes the error and average loss on the validation set and the test set. Display the 6 corresponding figures (error and average loss on train/valid/test), and write them in a log file.

### Part 10

> Train your network on the Fashion MNIST dataset. Plot the training/valid/test curves (error and loss as a function of the epoch number, corresponding to what you wrote in a file in the last question). Add to your report the curves obtained using your best hyperparameters, i.e. for which you obtained your best error on the validations et. We suggest 2 plots: the first one will plot the error rate (train/valid/test with different colors, show which color in a legend) and the other one for the averaged loss (on train/valid/test). You should be able to get less than 20% test error.